## Uniswap V3 Architecture (Math Explanation)

In [10]:
from rich import print
import numpy as np
import math

 we’re going to buy ETH for USDC at the price of $5000 per 1 ETH.
 
 We’ll need to find three ticks:

The current tick will correspond to the current price (5000 USDC for 1 ETH).
The lower and upper bounds of the price range we’re providing liquidity into. Let the lower price be $4545 and the upper price be $5500.

In [11]:
# Calculating price
def calculating_price(x:float, y:float) -> float:
    return math.sqrt(y, x)

# Calculating price tick
def price_to_tick(p:float) -> float:
    return math.floor(math.log(p, 1.0001))

'''
The way that UniswaV3 store sqrt(p) is using Q64.96 number which is the fixed point.
in previous calculation, the prices are floating point numbers, so we need to convert them to Q64.96 indeed.
Ultimately the result come from (price * (2**96)) just like following function:
'''
q96 = 2**96
def price_to_sqrtp(p:float) -> int:
    return int(math.sqrt(p) * q96)

# Liquidity Amount Calculation
sqrtp_low = price_to_sqrtp(4545)
sqrtp_cur = price_to_sqrtp(5000)
sqrtp_upp = price_to_sqrtp(5500)



'''
Based on the noted calculation, for calculating liquidity, there are two different segment for
each pair, once liquidity got x and once liquidity for y.
'''
def liquidity0(amount:float, pa:float, pb:float) -> float:
    '''
    amount -> amount of liquidity to add
    pa -> current price Q64.96
    ob -> upper price (in specific range) Q64.96
    '''
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    '''
    amount -> amount of liquidity to add
    pc -> current price Q64.96
    ob -> upper price (in specific range) Q64.96
    '''
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

eth_precision = 10**18
amount_eth = 1 * eth_precision
amount_usdc = 5000 * eth_precision

liq0 = liquidity0(amount_eth, sqrtp_cur, sqrtp_upp)
liq1 = liquidity1(amount_usdc, sqrtp_cur, sqrtp_low)
liq = int(min(liq0, liq1))
# > 1517882343751509868544




<br />





### Token Amounts Calculation
Since we choose the amounts we’re going to deposit, the amounts can be wrong. We cannot deposit any amounts at any price range; the liquidity amount needs to be distributed evenly along the curve of the price range we’re depositing into. Thus, even though users choose amounts, the contract needs to re-calculate them, and actual amounts will be slightly different (at least because of rounding).

Luckily, we already know the formulas:

In [12]:
# Token Amounts Calculation
def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)

def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

amount0 = calc_amount0(liq, sqrtp_upp, sqrtp_cur)
amount1 = calc_amount1(liq, sqrtp_low, sqrtp_cur)
(amount0, amount1)
# > (998976618347425408, 5000000000000000000000)

(998976618347425408, 5000000000000000000000)

### Test Functions:

In [13]:
price_to_tick(5000)
sqrtp_cur

5602277097478614198912276234240

<hr />

## First Swap
After deciding how many tokens we want to sell, we need to calculate how many tokens we’ll get in exchange. In Uniswap V2, we would’ve used current pool reserves, but in Uniswap V3 we have L and sqrt(P) and we know the fact that when swapping within a price range, only sqrt(P) changes and L remains unchanged (Uniswap V3 acts exactly as V2 when swapping is done only within one price range). We also know that:

In [14]:
amount_in = 42 * eth_precision
price_diff = (amount_in * q96) // liq
price_next = sqrtp_cur + price_diff
print("New price:", (price_next / q96) ** 2)
print("New sqrtP:", price_next)
print("New tick:", price_to_tick((price_next / q96) ** 2))


New price: 5003.913912782393

New sqrtP: 5604469350942327889444743441197

New tick: 85184

After finding the target price, we can calculate token amounts using the amounts calculation functions from a previous chapter:

In [15]:
amount_in = calc_amount1(liq, price_next, sqrtp_cur)
amount_out = calc_amount0(liq, price_next, sqrtp_cur)

print("USDC in:", amount_in / eth_precision)
print("ETH out:", amount_out / eth_precision)

USDC in: 42.0

ETH out: 0.008396714242162444